In [1]:
from typing import *
import wafel
import ctypes as C
import struct
from itertools import chain, combinations, product
from random import *
import numpy as np
import copy

In [2]:
%run utils.ipynb

In [3]:
# ---general config---

# path to unlocked dll (unlock through wafel)
game = wafel.Game('sm64_{0}.dll'.format(VERSION))
# path to initial m64
filename = 'm64s/best_found.m64'
# Output filename if an improvement is found
# Can set it to overwrite the old file if you don't mind it being overwritten
filename_new = 'm64s/best_found.m64'
# The m64 needs to be from start, but only want to brute force the joystick
# in a certain slice of time
# Note: This is 1 less than the frame number displayed in Mupen
# first affected frame is start_frame+1 (like after loading a st in mupen, you need to advance first)

start_frame = 77675
#end_frame = 77603
end_frame = 77694
#73494, 501, 508, 515, 522, 529, 536
# ---config end---

In [4]:
# Create a savestate slot
power_on = game.save_state()
m64 = wafel.load_m64(filename)

In [5]:
# Run through the m64 and make a savestate at the beginning of the bruteforce window.
# Also prints out number of stars just to make sure things are
# working/no desyncs
print(f'Fast forwarding to frame {start_frame}...')
for frame in range(len(m64[1])):
    set_inputs(game, m64[1][frame])
    game.advance()

    # print periodically
    if frame % 10000 == 0:
        print("Frame %05d stars %02d" % (frame, game.read('gMarioState.numStars')))
    if frame == start_frame:
        print("---bruteforce start---")
        startst = game.save_state()
        #break
        
    # Can keep this to examine actions if you want. Can be helpful as the actions are often obscure
    if (frame >= start_frame and frame <= end_frame + 2):
       print('{0} {1}'.format(frame, game.read('gMarioState.action')))

    # Example of making a savestate
    # game.save_state(backup)
    # backupFrame = frame + 1
    # break

Fast forwarding to frame 77675...
Frame 00000 stars 00
Frame 10000 stars 02
Frame 20000 stars 09
Frame 30000 stars 16
Frame 40000 stars 22
Frame 50000 stars 31
Frame 60000 stars 36
Frame 70000 stars 39
---bruteforce start---
77675 8914006
77676 16779404
77677 8914006
77678 16779404
77679 67110001
77680 67110001
77681 67110001
77682 67110001
77683 67109952
77684 8914006
77685 16779404
77686 67110001
77687 67110001
77688 67110001
77689 67110001
77690 67109952
77691 67109952
77692 8914006
77693 16779404
77694 67110001
77695 67110001
77696 67110001


In [6]:
# The DLL symbols have to be found through decomp (in object_fields.h)
# e.g. mario_action = game.read('gMarioState.action')

# Make a backup of the original inputs
m64_orig = copy.copy(m64)

In [7]:
def l1ofdiff(m64):
    '''Can include this in your fitness function for regularization if you want.
    Will penalize difference between joystick numbers on consecutive frames.'''
    tot = 0
    for i in range(start_frame, end_frame + 1):
        tot += abs(to_actual_coord(m64[1][i].stick_x) -
                   to_actual_coord(m64[1][i + 1].stick_x))
        tot += abs(to_actual_coord(m64[1][i].stick_y) -
                   to_actual_coord(m64[1][i + 1].stick_y))
    return tot

In [8]:
def dist_to_line(x1, z1, x2, z2, mario_x, mario_z):
    '''Computes distance from (mario_x, mario_z) to the line (x1,z1),(x2,z2).'''
    return abs((x2-x1)*(z1-mario_z)-(x1-mario_x)*(z2-z1))/(((x2-x1)**2+(z2-z1)**2)**.5)

def opt_func():
    '''Specify the function you wish to optimize here. The optimizer will attempt to minimize it.'''
    
    # Various potentially useful Mario variables
    mario_pos = game.read('gMarioState.pos')
    hspd = game.read('gMarioState.forwardVel')
    slideyaw = (game.read('gMarioState.slideYaw')) % 65536
    slidevel_x = (game.read('gMarioState.slideVelX'))
    slidevel_z = (game.read('gMarioState.slideVelZ'))
    yspd = game.read('gMarioState.vel')[1]
    mario_yaw = game.read('gMarioState.faceAngle')[1]
    mario_action = game.read('gMarioState.action')
    num_coins = game.read('gMarioState.numCoins')
    
    #Some miscellaneous examples
    return (-mario_pos[1]) + (hspd < 38.5)*10000 + (yspd != -4.0)*1000
    #return ((mario_pos[0] - 3946)**2 + (mario_pos[2] - 2117)**2)**.5 + (hspd != 0)*10000 + (mario_yaw + 30000)*.001
    #return (mario_pos[1] < -1005)*500 - hspd + abs(mario_yaw - 23400)*.002
    #return ((mario_pos[0] + 200)**2+(mario_pos[2]-3461)**2)**.5+abs(mario_pos[1]+2051)*100000+abs(hspd<43)*10000
    #return (hspd < 50)*1000 + abs(mario_yaw + 5600)*.3 + ((mario_pos[0] + 3231)**2 + (mario_pos[2] + 1794)**2)**.5
    #second monty mole
    #return ((mario_pos[0]+2525)**2 + (mario_pos[2] + 2626)**2)**.5 + (mario_pos[1] < -2507)*30
    #ttm lonely mushroom land by crazy box
    #return ((mario_pos[0]-5021)**2+(mario_pos[2]-2780)**2)**.5
    #slip slidin away at wall clip time
    #return -hspd*.2 + (hspd<26)*10000 + (mario_pos[2] > -6490)*20000 + mario_pos[0]*5 + (mario_pos[2]+6500)**2*.2
    #slip slidin away dist to door
    #return ((mario_pos[0]+7628)**2+(mario_pos[2]+6988)**2)**.5
    


In [9]:
#This cell is definitely the messiest, and where you might want to do something
#different. Roughly speaking, it does annealing with random restarts.

#keep track of best m64 found and best objective value
best_ever_m64 = copy.copy(m64_orig)
best_ever_val = 9999999999

# Can populate excluded_frames with frame numbers to disable perturbations during e.g. a cutscene
excluded_frames = []
all_frames = list(range(start_frame + 1, end_frame + 1))
allowed_frames = [f for f in all_frames if f not in excluded_frames]
    
for attempt in range(500000):
    #Each attempt is a random restart
    
    last_change = 0
    #Temperature for annealing
    temp = random()*1.5
    #Determine the size of joystick perturbations and number of joystick perturbations
    max_changes = randint(1, 8)
    max_size = randint(1, 30)
    #With some probability, drop down to size 1 or count 1 (helps explore locally)
    big_change_num_prob = random()*.2
    big_change_size_prob = random()*.5
    m64 = (best_ever_m64[0], copy.copy(best_ever_m64[1]))
    best_val = 9999999999
    cur_val = best_val
    
    #Now try random perturbations
    for i in range(500000):
        #Break out early if these settings get stuck
        if i - last_change > 3000:
            print(temp)
            print(max_changes)
            print(max_size)
            break
        #Array to keep track of changes made to the m64.
        #Make some random changes, check fitness, revert if not good enough.
        changes = []
        if i % 1000 == 0:
            print(str(i) + ' ' + str(cur_val))
        #Lower temperature
        if i % 300 == 0:
            temp = temp*.8
        #Do a perturbation here, except for first time through
        #where we just want to get objective value
        if i > 0:
            num_changes = randint(1, max_changes)
            if random() > big_change_num_prob and i > 3000:
                    num_changes = 1
            for change in range(num_changes):
                #Randomly select change frame (with replacement in
                #the event of multiple changes)
                change_frame = choice(allowed_frames)#randint(start_frame + 1, end_frame)
                change_dir = randint(0, 3)
                change_size = randint(1, max_size)
                if random() > big_change_size_prob and i > 3000:
                    change_size = 1
                #Make sure to save old version of frame so we can revert
                frame_old = m64[1][change_frame].copy()
                changes.append([change_frame, frame_old])
                #Modify joystick input in the appropriate size and direction
                for change in range(change_size):
                    frame_inp = m64[1][change_frame].copy()
                    # print(frame_inp.stick_y)
                    if change_dir == 0:
                        frame_inp.stick_x = increment_coord(frame_inp.stick_x)
                    elif change_dir == 1:
                        frame_inp.stick_y = increment_coord(frame_inp.stick_y)
                    elif change_dir == 2:
                        frame_inp.stick_x = 255 - increment_coord(255 - frame_inp.stick_x)
                    else:
                        frame_inp.stick_y = 255 - increment_coord(255 - frame_inp.stick_y)
                    m64[1][change_frame] = frame_inp

        #now find its fitness to minimize
        game.load_state(startst)
        offset = 0
        for frame in range(start_frame, end_frame):
            set_inputs(game, (m64[1][frame + 1]))
            game.advance()
        x, y, z = game.read('gMarioState.pos')
        hspd = game.read('gMarioState.forwardVel')
        faceyaw = game.read('gMarioState.faceAngle')[1]
        orig_fitness = opt_func()#to_hmc_bounce_before_elevator()#to_hmc_elevator()
        fitness = orig_fitness + l1ofdiff(m64)*.003#+ l1ofdiff(m64)*5# + xnorm(m64)*7
        if fitness < cur_val:
            last_change = i
            cur_val = fitness
            if fitness < best_val:
                print('New best: {0} {1}'.format(fitness, orig_fitness))
                print('{0} {1} {2} {3} {4}'.format(x, y, z, hspd, faceyaw))
                best_val = fitness
            if fitness < best_ever_val:
                wafel.save_m64(filename_new, m64[0], m64[1])
                best_ever_val = fitness
                best_ever_m64 = (m64[0], copy.copy(m64[1]))
        #Chose the most basic function for annealing
        elif random() < np.exp(-(fitness - cur_val)/temp):
        #elif random() < np.exp(-(fitness - best_val)/temp):
            if fitness != cur_val:
                last_change = i
            cur_val = fitness
        else:
            #if we failed, revert m64
            #revert changes in reverse order of how they were made
            for change in changes[::-1]:
                m64[1][change[0]] = change[1]

0 9999999999
New best: 415.699234375 412.990234375
3729.119873046875 -412.990234375 1874.0302734375 40.1229248046875 -25144
New best: 413.79772119140625 411.07672119140625
3728.994140625 -411.07672119140625 1874.1708984375 40.039085388183594 -25144
1000 415.0857102050781
2000 414.87172119140627
3000 414.13372119140627
New best: 413.7917211914062 411.07672119140625
3728.85400390625 -411.07672119140625 1874.3128662109375 40.10208511352539 -25360
New best: 413.68972119140625 411.07672119140625
3728.824462890625 -411.07672119140625 1874.4051513671875 40.09075927734375 -25360
New best: 413.6837211914063 411.07672119140625
3728.824462890625 -411.07672119140625 1874.4051513671875 40.09075927734375 -25360
4000 413.73772119140625
New best: 413.67772119140625 411.07672119140625
3728.8125 -411.07672119140625 1874.2694091796875 40.04365158081055 -25331
New best: 413.6717211914063 411.07672119140625
3728.9775390625 -411.07672119140625 1874.376953125 40.034515380859375 -25360
New best: 413.665721191

New best: 412.8162509765625 410.5572509765625
3728.922607421875 -410.5572509765625 1873.962890625 40.074256896972656 -25387
New best: 412.8102509765625 410.5572509765625
3728.941650390625 -410.5572509765625 1873.9910888671875 40.06932067871094 -25387
New best: 412.7982509765625 410.5572509765625
3728.95263671875 -410.5572509765625 1873.979736328125 40.07595443725586 -25387
New best: 412.7922509765625 410.5572509765625
3728.960693359375 -410.5572509765625 1873.958984375 40.078346252441406 -25387
New best: 412.7862509765625 410.5572509765625
3728.961181640625 -410.5572509765625 1873.9619140625 40.084983825683594 -25387
New best: 412.7802509765625 410.5572509765625
3728.949462890625 -410.5572509765625 1873.9688720703125 40.08443069458008 -25387
New best: 412.7742509765625 410.5572509765625
3728.945556640625 -410.5572509765625 1873.9713134765625 40.08448791503906 -25387
New best: 412.7682509765625 410.5572509765625
3728.92236328125 -410.5572509765625 1873.9869384765625 40.08713150024414 -2

New best: 412.1862509765625 410.5572509765625
3728.9306640625 -410.5572509765625 1873.98583984375 40.130943298339844 -25319
7000 412.1862509765625
New best: 412.1802509765625 410.5572509765625
3728.97705078125 -410.5572509765625 1873.9404296875 40.1338996887207 -25338
New best: 412.1742509765625 410.5572509765625
3728.999755859375 -410.5572509765625 1873.978515625 40.1274299621582 -25338
New best: 412.1682509765625 410.5572509765625
3728.881591796875 -410.5572509765625 1873.9268798828125 40.13481140136719 -25338
New best: 412.1622509765625 410.5572509765625
3728.89599609375 -410.5572509765625 1873.912353515625 40.141357421875 -25338
New best: 412.1562509765625 410.5572509765625
3728.915771484375 -410.5572509765625 1873.9937744140625 40.1236572265625 -25338
New best: 412.1502509765625 410.5572509765625
3728.93359375 -410.5572509765625 1873.977294921875 40.129329681396484 -25338
New best: 412.1442509765625 410.5572509765625
3728.904296875 -410.5572509765625 1873.9132080078125 40.14117813

New best: 411.7902509765625 410.5572509765625
3728.902587890625 -410.5572509765625 1873.8897705078125 40.17882537841797 -25367
New best: 411.7842509765625 410.5572509765625
3728.9306640625 -410.5572509765625 1873.861328125 40.18520736694336 -25367
New best: 411.7782509765625 410.5572509765625
3728.954345703125 -410.5572509765625 1873.8399658203125 40.19013977050781 -25367
New best: 411.7722509765625 410.5572509765625
3728.969482421875 -410.5572509765625 1873.825439453125 40.19404220581055 -25367
New best: 411.7662509765625 410.5572509765625
3728.939453125 -410.5572509765625 1873.854248046875 40.19057083129883 -25367
New best: 411.7602509765625 410.5572509765625
3728.962158203125 -410.5572509765625 1873.832763671875 40.19565963745117 -25367
New best: 411.7542509765625 410.5572509765625
3728.97265625 -410.5572509765625 1873.8221435546875 40.19878005981445 -25367
New best: 411.7482509765625 410.5572509765625
3728.98681640625 -410.5572509765625 1873.8095703125 40.202266693115234 -25367
100

New best: 411.4082619628906 410.0372619628906
3728.932373046875 -410.0372619628906 1872.69580078125 40.54479217529297 -25001
New best: 411.40226196289063 410.0372619628906
3728.948974609375 -410.0372619628906 1872.6800537109375 40.546104431152344 -25001
New best: 411.3962619628906 410.0372619628906
3728.9716796875 -410.0372619628906 1872.6578369140625 40.55064392089844 -25001
New best: 411.39026196289063 410.0372619628906
3728.9716796875 -410.0372619628906 1872.6578369140625 40.55064392089844 -25001
New best: 411.3842619628906 410.0372619628906
3728.950927734375 -410.0372619628906 1872.67724609375 40.5466423034668 -25001
New best: 411.37826196289063 410.0372619628906
3728.950927734375 -410.0372619628906 1872.67724609375 40.5466423034668 -25001
New best: 411.3722619628906 410.0372619628906
3728.98486328125 -410.0372619628906 1872.763671875 40.52960205078125 -25020
7000 411.3722619628906
New best: 411.36626196289063 410.0372619628906
3728.96484375 -410.0372619628906 1872.6746826171875 40

7000 411.3602619628906
8000 411.27026196289063
9000 411.2462619628906
10000 411.2162619628906
11000 411.1682619628906
12000 411.13226196289065
13000 411.13226196289065
14000 411.13226196289065
7.096257915339784e-06
5
28
0 9999999999
New best: 411.07226196289065 410.0372619628906
3728.923095703125 -410.0372619628906 1872.4068603515625 40.467708587646484 -25304
1000 413.0522619628906
2000 413.9262509765625
3000 413.25026196289065
4000 413.0762619628906
5000 412.63226196289065
6000 412.28426196289064
7000 411.6842619628906
8000 411.57026196289064
9000 411.53426196289064
10000 411.53426196289064
11000 411.53426196289064
12000 411.53426196289064
0.00010874123211864036
2
18
0 9999999999
New best: 411.07226196289065 410.0372619628906
3728.923095703125 -410.0372619628906 1872.4068603515625 40.467708587646484 -25304
1000 413.7462509765625
2000 413.8662509765625
3000 412.90226196289063
4000 412.6862619628906
5000 412.5662619628906
6000 412.46426196289065
7000 412.16426196289063
8000 411.67826196

1000 412.7402619628906
2000 412.1342619628906
3000 411.4322619628906
4000 411.2162619628906
5000 411.1502619628906
6000 411.0062619628906
7000 410.96426196289065
8000 410.92826196289064
New best: 410.9222619628906 410.0372619628906
3728.932373046875 -410.0372619628906 1872.1448974609375 40.51205062866211 -25735
9000 410.9222619628906
10000 410.9222619628906
11000 410.9222619628906
6.247019347466531e-05
6
29
0 9999999999
New best: 410.9222619628906 410.0372619628906
3728.932373046875 -410.0372619628906 1872.1448974609375 40.51205062866211 -25735
1000 411.9842619628906
2000 412.90226196289063
3000 412.20026196289064
4000 412.35626196289064
5000 411.8042619628906
6000 411.48626196289064
7000 411.14426196289065
8000 411.0542619628906
9000 411.02426196289065
10000 411.02426196289065
11000 411.02426196289065
12000 411.0182619628906
13000 411.0182619628906
14000 411.01226196289065
15000 411.01226196289065
16000 411.01226196289065
5.548293407988696e-06
8
20
0 9999999999
New best: 410.922261962

New best: 410.1521884765625 408.6431884765625
3727.947509765625 -408.6431884765625 1873.35498046875 39.91852569580078 -25735
New best: 410.1461884765625 408.6431884765625
3727.942626953125 -408.6431884765625 1873.3643798828125 39.91728210449219 -25735
New best: 410.1401884765625 408.6431884765625
3727.927734375 -408.6431884765625 1873.347412109375 39.92206954956055 -25735
New best: 410.1341884765625 408.6431884765625
3727.928955078125 -408.6431884765625 1873.3828125 39.90250015258789 -25735
New best: 410.1281884765625 408.6431884765625
3727.928955078125 -408.6431884765625 1873.3828125 39.90250015258789 -25735
New best: 410.1221884765625 408.6431884765625
3727.904052734375 -408.6431884765625 1873.3779296875 39.90864944458008 -25735
New best: 410.1101884765625 408.6431884765625
3727.904052734375 -408.6431884765625 1873.3779296875 39.90864944458008 -25735
New best: 410.1041884765625 408.6431884765625
3727.916748046875 -408.6431884765625 1873.3631591796875 39.921470642089844 -25735
New bes

2000 411.4661884765625
3000 410.8901884765625
4000 410.5541884765625
5000 410.3261884765625
6000 409.5961689453125
7000 409.5121689453125
8000 409.4881689453125
9000 409.4821689453125
10000 409.4821689453125
11000 409.4821689453125
0.00016418626605424429
8
16
0 9999999999
New best: 409.4341689453125 408.1231689453125
3727.990234375 -408.1231689453125 1872.9971923828125 40.27663040161133 -25601
1000 413.3116281738281
2000 413.7401884765625
3000 412.4101689453125
4000 412.0801689453125
5000 411.0241689453125
6000 410.4121689453125
7000 410.1061689453125
8000 409.9381689453125
9000 409.8661689453125
10000 409.8541689453125
11000 409.8541689453125
12000 409.8541689453125
8.652740210340166e-05
5
13
0 9999999999
New best: 409.4341689453125 408.1231689453125
3727.990234375 -408.1231689453125 1872.9971923828125 40.27663040161133 -25601
1000 413.4222509765625
2000 414.4422509765625
3000 414.75172119140626
4000 413.1076281738281
5000 412.6756281738281
6000 411.8621884765625
7000 411.172188476562

KeyboardInterrupt: 